In [1]:
import dask
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import ast
import collections
import math
import re

In [2]:
parquetdir = '../../tcm-columns-add-main/'

In [3]:
# revision_text= ddf.read_parquet(parquetdir + 'revision.text').compute()

In [6]:
page_title = pd.read_parquet(parquetdir + 'page.title')

In [7]:
links = pd.read_parquet('../../intermediate-result/TCM/wikilink-info')

In [8]:
# convert string to list of wikilinks
links['wikilinks'] = links['wikilinks'].map(lambda x: ast.literal_eval(x.lower()))

In [10]:
page_texts = page_title.join(links)

In [11]:
page_texts = page_texts.groupby('page.title')

In [13]:
def get_link_title(links):
    # remove the "[[" and "]]" and split the links
    return [l[2:-2].split('|')[0] for l in links]

In [14]:
%%time
page_wlinkcounts={page: page_texts.get_group(page)['wikilinks'].apply(
    lambda x: collections.Counter(get_link_title(x))
    ).sum() 
for page in  page_texts.groups.keys() }

CPU times: user 1min 28s, sys: 1.1 s, total: 1min 29s
Wall time: 1min 29s


In [18]:
month = ['january','february','march','april','may','june','july','august','september','october','november','december']

In [19]:
def filter_links(counter):
    removed_keys = [i for i in counter if re.match(r'^([\s\d]+)$', i) or any(xs in i for xs in month) or re.match('\d{4}[-/]\d{2}[-/]\d{2}',i)]
    for i in removed_keys:
        del counter[i]
    return counter

In [20]:
for k,v in page_wlinkcounts.items():
    filter_links(v)

In [17]:
# page_wlinkcounts

In [18]:
def cosine_similarity(counter1,counter2):
    c1squared= sum([a*a for a in counter1.values()])
    c2squared= sum([a*a for a in counter2.values()])
    c1c2=sum([counter1[k]*counter2[k]  for k in counter1.keys() if k in counter2])
    return c1c2/(math.sqrt(c1squared*c2squared))

In [19]:
%%time
page_wlink_similarities={
    key1:{
        k:v for k,v in {
            key2: cosine_similarity(page_wlinkcounts[key1],page_wlinkcounts[key2]) 
            for key2 in page_wlinkcounts.keys() 
            if not key1.startswith('Talk:') and not key2.startswith('Talk:') 
                 and page_wlinkcounts[key1] and page_wlinkcounts[key2]
            }.items() if 1>v>0.02
        } for key1 in page_wlinkcounts.keys()}

CPU times: user 8.23 s, sys: 1.96 ms, total: 8.23 s
Wall time: 8.23 s


In [20]:
df = pd.DataFrame.from_dict(page_wlink_similarities,orient='index')

In [21]:
df = df.fillna(0)

In [22]:
df = df[df.index.to_list()]

In [23]:
np.fill_diagonal(df.values, 1)

In [24]:
df.to_csv("article-similarity-matrix.csv", sep="\t",encoding='utf-16')

In [25]:
df.to_parquet('article-similarity-matrix')